In [ ]:
!pip install dpt

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

In [ ]:
import dpt
from PIL import Image
import requests
import torch
import numpy as np
from transformers import DPTFeatureExtractor, DPTForDepthEstimation

In [ ]:
# Create a DPT feature extractor
feature_extractor = DPTFeatureExtractor.from_pretrained("Intel/dpt-large")

# Create a DPT depth estimation model
model = DPTForDepthEstimation.from_pretrained("Intel/dpt-large")

In [ ]:
# Specify the path to the image file
image_path = "D:/minie/code/nyu_data/data/nyu2_train/bathroom_0002_out/34.jpg"

# Open the image using PIL
image = Image.open(image_path)

pixel_values = feature_extractor(image, return_tensors="pt").pixel_values

In [ ]:
# Use torch.no_grad() to disable gradient computation
with torch.no_grad():
    # Pass the pixel values through the model
    outputs = model(pixel_values)
    # Access the predicted depth values from the outputs
    predicted_depth = outputs.predicted_depth

In [ ]:
# Interpolate the predicted depth values to the original size
prediction = torch.nn.functional.interpolate(
    predicted_depth.unsqueeze(1),
    size=image.size[::-1],
    mode="bicubic",
    align_corners=False,
).squeeze()

# Convert the interpolated depth values to a numpy array
output = prediction.cpu().numpy()

# Scale and format the depth values for visualization
formatted = (output * 255 / np.max(output)).astype('uint8')

# Create an image from the formatted depth values
depth = Image.fromarray(formatted)
depth